In [1]:
import pandas as pd
import numpy as np 
import json
import os

In [3]:
read = pd.read_json("raw/out_label/summaries_out2.json", lines = True)
df = read[read.answer == "accept"]
df.head()

,text,article_id,_input_hash,_task_hash,_is_binary,spans,tokens,_view_id,relations,answer,_timestamp
0,Ukraine war: Russian tactics on eastern front ...,0,1192826571,1725000051,False,"[{'start': 0, 'end': 7, 'token_start': 0, 'tok...","[{'text': 'Ukraine', 'start': 0, 'end': 7, 'id...",relations,"[{'head': 3, 'child': 0, 'head_span': {'start'...",accept,1666853824
2,Lee Jae-yong: Samsung appoints Lee Jae-yong to...,2,-386898449,495824270,False,"[{'start': 0, 'end': 12, 'token_start': 0, 'to...","[{'text': 'Lee', 'start': 0, 'end': 3, 'id': 0...",relations,"[{'head': 31, 'child': 18, 'head_span': {'star...",accept,1666854104
3,New Zealand Instagram couple 'relieved' after ...,3,94784817,-1047716132,False,"[{'start': 0, 'end': 28, 'token_start': 0, 'to...","[{'text': 'New', 'start': 0, 'end': 3, 'id': 0...",relations,"[{'head': 3, 'child': 9, 'head_span': {'start'...",accept,1666854141
4,Iran protests: Police fire on Mahsa Amini mour...,4,-1204324385,1067222802,False,"[{'start': 0, 'end': 4, 'token_start': 0, 'tok...","[{'text': 'Iran', 'start': 0, 'end': 4, 'id': ...",relations,"[{'head': 3, 'child': 8, 'head_span': {'start'...",accept,1666854246
5,Germany plans to legalise recreational cannabi...,5,1523244431,980527859,False,"[{'start': 0, 'end': 7, 'token_start': 0, 'tok...","[{'text': 'Germany', 'start': 0, 'end': 7, 'id...",relations,"[{'head': 0, 'child': 5, 'head_span': {'start'...",accept,1666854286


In [5]:
df.shape

(323, 11)

In [7]:
#get data into needed format and create new DataFrame
data = []
for index,row in df.iterrows():
    rel_list = []
    for rel in row.relations:
        subj = row.text[rel["head_span"]["start"]:rel["head_span"]["end"]]
        obj = row.text[rel["child_span"]["start"]:rel["child_span"]["end"]]
        rel_list.append(f"<triplet> {subj} <subj> {obj} <obj> {rel['label']}")
    data.append({"doc_id":row.article_id, "text": row.text, "triplets": " ".join(rel_list)})

data = pd.DataFrame(data, columns = ["doc_id","text","triplets"])

In [13]:
data = data[data.triplets != ""] #filter empty relations

In [18]:
data["rel_count"] = data.triplets.apply(lambda x: x.count("<triplet>"))

C:\Users\svawe\AppData\Local\Temp/ipykernel_3112/2853497938.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["rel_count"] = data.triplets.apply(lambda x: x.count("<triplet>"))


In [20]:
data.groupby("rel_count").doc_id.count()

rel_count
1    76
2    83
3    54
4    25
5    16
6     9
7     8
8     2
Name: doc_id, dtype: int64

In [23]:
data["prop"] = data.text.apply(lambda x: "'s" in x)

C:\Users\svawe\AppData\Local\Temp/ipykernel_3112/95724760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["prop"] = data.text.apply(lambda x: "'s" in x)


In [31]:
import re
re.split("<triplet>", e1)

['',
 ' presidential pardon <subj> Lee Jae-yong <obj> ProvideAid ',
 ' Samsung <subj> Lee Jae-yong <obj> EngageInMaterialCooperation']

In [32]:
e1 = ['<triplet> presidential pardon <subj> Lee Jae-yong <obj> ProvideAid <triplet> Samsung <subj> Lee Jae-yong <obj> EngageInMaterialCooperation']

In [40]:
d = "<triplet>"
len_list = []
trip_list = []
for triplet in data.triplets.to_list():
    s =  [d+e.rstrip() for e in triplet.split(d) if e]
    trip_list.append(s)
    len_list.append(len(s))

In [42]:
data["len"] = len_list

C:\Users\svawe\AppData\Local\Temp/ipykernel_3112/1903591471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["len"] = len_list


In [45]:
#dedupe = [list(set(x)) for x in trip_list]
new_len = [len(list(set(x))) for x in trip_list]
data["new_len"] = new_len

C:\Users\svawe\AppData\Local\Temp/ipykernel_3112/2558941106.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["new_len"] = new_len


In [47]:
data[data.len != data.new_len]

,doc_id,text,triplets,rel_count,prop,len,new_len
14,17,Ukraine war: Russia troop deployment to Belaru...,<triplet> Russia <subj> Ukraine <obj> Fight <t...,3,False,3,2
29,36,"Giuseppe Conte: ""If Meloni Pursues Reactionary...",<triplet> Giuseppe Conte <subj> Meloni <obj> M...,5,True,5,4
40,49,Russia's Nuclear Threat: Is Russia's Bluffing ...,<triplet> Russia <subj> Ukraine <obj> Fight <t...,3,True,3,2
43,53,IEA expects global emissions to peak in 2025. ...,<triplet> Russian <subj> Ukraine <obj> Fight <...,3,False,3,2
45,55,Clashes erupt between Iranian security forces ...,<triplet> human rights group <subj> Iranian se...,4,True,4,3
47,57,UN says world’s climate pledges still ‘nowhere...,<triplet> UN <subj> world <obj> MakePublicStat...,3,True,3,1
48,59,Several people killed as gunmen open fire at s...,<triplet> gunmen <subj> Several people <obj> F...,7,False,7,6
52,64,Italian PM Meloni easily wins 2nd and final co...,<triplet> confidence vote <subj> Italian PM Me...,2,False,2,1
55,69,Adidas ends partnership with Kanye West over a...,<triplet> Adidas <subj> Kanye West <obj> Reduc...,2,True,2,1
64,80,Sunni extremists behind attack on mausoleum in...,<triplet> the Tasnim’s source <subj> Sunni ext...,5,False,5,4


In [25]:
data[data.prop == True]

,doc_id,text,triplets,rel_count,prop
2,3,New Zealand Instagram couple 'relieved' after ...,<triplet> New Zealand Instagram couple <subj> ...,1,True
3,4,Iran protests: Police fire on Mahsa Amini mour...,<triplet> Police <subj> Mahsa Amini mourners <...,2,True
9,11,The impact of Iran's protests beyond Iran's's ...,<triplet> Iranian Revolutionary Guard <subj> I...,1,True
16,19,North Korea tensions: Why is Kim Jong-un uppin...,<triplet> Kim Jong-un <subj> the Koreas <obj> ...,1,True
18,21,Kanye West: What's next for Yeezy and trainer ...,<triplet> Adidas <subj> Kanye West <obj> Reduc...,2,True
...,...,...,...,...,...
307,407,Ukraine: UNESCO using satellites to track cult...,<triplet> Russian <subj> Ukraine <obj> Fight <...,2,True
309,409,US to push for multinational Haiti interventio...,<triplet> US <subj> Canada <obj> Consult <trip...,5,True
310,411,US blames Russia's Wagner Group for worsening ...,<triplet> US <subj> Russia's Wagner Group <obj...,4,True
311,412,Putin’s speech at Valdai club to be studied cl...,<triplet> US expert <subj> Putin <obj> MakePub...,7,True
